# Angle recalculations

The angular deviation between two vectors will be reconstructed using two methods:
   - The acute angular method: Calculated by applying cosine.
   - The obtuse angular method: Calculated by applying atan2.
   
 

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
import math
warnings.simplefilter(action='ignore', category=FutureWarning)

### Load corrected integrated Pointing task

Data comes as a result of script `02_ParticipantsDataReconstruction`

In [2]:
CoordinatesIntegrated = pd.read_csv('E:/HumanA/Data/Data_Tracy/AgentPointingTask_Integrated.csv')
#CoordinatesIntegrated = pd.read_csv('/Users/tracysanchezpacheco/Documents/Resources/PointingTask_Integrated.csv')
CoordinatesIntegrated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2520 entries, 0 to 2519
Data columns (total 35 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     2520 non-null   int64  
 1   SubjectID                      2520 non-null   int64  
 2   ParticipantPosition_x          2520 non-null   float64
 3   ParticipantPosition_z          2520 non-null   float64
 4   PointerPosition_x              2520 non-null   float64
 5   PointerPosition_z              2520 non-null   float64
 6   PointerDirection_x             2520 non-null   float64
 7   PointerDirection_z             2520 non-null   float64
 8   AgentCenterWorld_z             2520 non-null   float64
 9   AgentCenterWorld_x             2520 non-null   float64
 10  DistanceToParticipant          2520 non-null   float64
 11  PointingTaskStartingLocations  2520 non-null   int64  
 12  AgentID                        2520 non-null   i

## Creation of vectors

In [3]:
CoordinatesIntegrated['vector_PartPos'] = list(zip(CoordinatesIntegrated.ParticipantPosition_x, CoordinatesIntegrated.ParticipantPosition_z))
CoordinatesIntegrated['vector_AgentPos'] = list(zip(CoordinatesIntegrated['AgentCenterWorld_x'], CoordinatesIntegrated['AgentCenterWorld_z']))
CoordinatesIntegrated['vector_PointerPos'] =list(zip(CoordinatesIntegrated.PointerDirection_x, CoordinatesIntegrated.PointerDirection_z))

In [4]:
PartList = list(CoordinatesIntegrated['vector_PartPos'])
BuiltList = list(CoordinatesIntegrated['vector_AgentPos'])
PointList = list(CoordinatesIntegrated['vector_PointerPos'])

In [5]:
### Calculating direction vector from Participant Position (a) to Target Agent Position (b): b- **  component form is terminal minus initial
CoordinatesIntegrated['Vector_Participant-Agent'] = [tuple(x-y for x, y in zip(tup1, tup2)) for tup1, tup2 in zip(BuiltList, PartList)]
### Calculating direction vector from Participant Position (a) to pointer (b): a-b
CoordinatesIntegrated['Vector_Participant-Pointer'] = [tuple(x-y for x, y in zip(tup1, tup2)) for tup1, tup2 in zip(PointList, PartList)] # addition
CoordinatesIntegrated.iloc[:,-15:-8].head(5)

,Trials,TrialNumber,StartPointID.1,ID_for_StartingPosition,ImageName,TimeDelta,RT
0,46,1,1,1,16_SaAct_92_36,0 days 00:00:09.057460992,9.057461
1,46,67,12,1,16_SaAct_92_36,0 days 00:00:28.543825408,28.543825
2,46,110,19,2,16_SaAct_92_36,0 days 00:00:05.412355584,5.412356
3,46,127,22,1,16_SaAct_92_36,0 days 00:00:08.281718784,8.281719
4,46,3,1,3,16_SaAct_92_36,0 days 00:00:15.555894272,15.555894


In [6]:
CoordinatesIntegrated.groupby(['AgentCenterWorld_z','AgentCenterWorld_x',  'ImageName']).count()

,,,Unnamed: 0,SubjectID,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,PointerDirection_z,DistanceToParticipant,PointingTaskStartingLocations,...,TimeDelta,RT,TimeOut,TimeStampBegin,TimeStampEnd,vector_PartPos,vector_AgentPos,vector_PointerPos,Vector_Participant-Agent,Vector_Participant-Pointer
AgentCenterWorld_z,AgentCenterWorld_x,ImageName,,,,,,,,,,,,,,,,,,,,,
-253.040756,-6.122803,09_CmAAct_44_32,49,49,49,49,49,49,49,49,49,49,...,49,49,49,49,49,49,49,49,49,49
-199.902664,-39.956726,27_SaAct_94_37,35,35,35,35,35,35,35,35,35,35,...,35,35,35,35,35,35,35,35,35,35
-196.890427,-146.324509,42_CmA_16_45,41,41,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,41,41,41,41
-195.873672,96.517624,46_Sa_78_25,30,30,30,30,30,30,30,30,30,30,...,30,30,30,30,30,30,30,30,30,30
-167.546097,-192.775833,55_Sa_62_7,40,40,40,40,40,40,40,40,40,40,...,40,40,40,40,40,40,40,40,40,40
-150.303055,-55.071732,16_SaAct_92_36,41,41,41,41,41,41,41,41,41,41,...,41,41,41,41,41,41,41,41,41,41
-142.439651,146.622894,11_CmAAct_50_21,39,39,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
-138.621201,-303.645630,07_CmAAct_52_59,39,39,39,39,39,39,39,39,39,39,...,39,39,39,39,39,39,39,39,39,39
-137.772018,-194.664444,18_SaAct_102_48,45,45,45,45,45,45,45,45,45,45,...,45,45,45,45,45,45,45,45,45,45


## Angle functions acute method

In [7]:
def DotTuple(v1,v2):
    """Returns the dot product of two vectors of two elements each
Parameters
-------------
    v1: tuple of two elements
    v2: tuple of two elements
returns
----------
    list of dot product of tuples
"""
    dotList = []
    dot = [sum(x*y for x, y in zip(tup1, tup2))for tup1, tup2 in zip(v1, v2)]
    dotList.append(dot)
    flatlist = [item for sublist in dotList for item in sublist]
    return flatlist

In [8]:
def Magnitud(v1):
    """Returns the magnitute of a vector
Parameters
-------------
    v1: tuple of two elements
returns
----------
    list with magnitude of vectors
"""
    v1 = [x ** 2 + y ** 2 for x,y in v1]
    squared = [math.sqrt(value) for value in v1]
    return(squared)

In [9]:
CoordinatesIntegrated['Agent-Participant_x'] = CoordinatesIntegrated['AgentCenterWorld_x'] -  CoordinatesIntegrated['ParticipantPosition_x']
CoordinatesIntegrated['Agent-Participant_z'] = CoordinatesIntegrated['AgentCenterWorld_z'] - CoordinatesIntegrated['ParticipantPosition_z']

In [12]:
# Creating the individual elements of the arc cos formula and inserting them on the dataframe
CoordinatesIntegrated['Dot_Pointer-Agent-Participant'] = DotTuple(CoordinatesIntegrated['Vector_Participant-Pointer'],CoordinatesIntegrated['Vector_Participant-Agent'])
CoordinatesIntegrated['Magnitud_Participant-Pointer'] = Magnitud(CoordinatesIntegrated['Vector_Participant-Pointer'])
CoordinatesIntegrated['Magnitud_Participant-Agent'] = Magnitud(CoordinatesIntegrated['Vector_Participant-Agent'])

In [13]:
# Calculating degrees using the arc cos formula and inserting them on the dataframe
my_angle = CoordinatesIntegrated['Dot_Pointer-Agent-Participant']/ (CoordinatesIntegrated['Magnitud_Participant-Pointer'] * CoordinatesIntegrated['Magnitud_Participant-Agent'])
CoordinatesIntegrated['radians'] =  [math.acos(value) for value in my_angle]
CoordinatesIntegrated['degrees'] = [np.rad2deg(value) for value in CoordinatesIntegrated['radians']]
CoordinatesIntegrated['degrees'].describe()

count    2520.000000
mean       41.602468
std        37.503039
min         0.037487
25%        13.196281
50%        30.475529
75%        59.371632
max       178.950243
Name: degrees, dtype: float64

## Angle functions obtuse method

In [14]:
def signedAngle(v1, v2):
    """" Returns the subtraction of arc tangent of vector 2 minus vector 1
Parameters
-------------
v1: tuple of values that contains the coordinates of your reference initial vector,
    in our case the perfect angle between position and target (remember to order them y,x)
v2: tuple of values that contains the coordinates of your vector of contrast,
    in our case the real reply from the participants (remember to order them y,x)
returns
----------

"""
    angle = math.atan2(v2[1], v2[0]) - math.atan2(v1[1], v1[0])
    return(np.rad2deg(angle))

In [15]:
signedAngles = []

for count, x in enumerate(CoordinatesIntegrated['PointerDirection_z']):
    signedAngles = signedAngles + [signedAngle([ CoordinatesIntegrated['PointerDirection_z'][count], CoordinatesIntegrated['PointerDirection_x'][count]], [CoordinatesIntegrated['Agent-Participant_z'][count]
       , CoordinatesIntegrated['Agent-Participant_x'][count]])]

# Account for angle deviations > 180°
for count, angle in enumerate(signedAngles):
    if angle < 0:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle360", signedAngles, True)

for count, angle in enumerate(signedAngles):
    if angle > 180:
        signedAngles[count] = angle - 360
    elif angle < -180:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle-+180", signedAngles, True)
CoordinatesIntegrated['SignedAngle-+180'].describe()

count    2520.000000
mean       -8.042055
std        72.683654
min      -179.910652
25%       -52.129808
50%        -6.902065
75%        33.069170
max       179.548332
Name: SignedAngle-+180, dtype: float64

In [16]:
PointerZ = CoordinatesIntegrated.loc[:,'PointerDirection_z'].values
PointerX = CoordinatesIntegrated.loc[:,'PointerDirection_x'].values
AgentParticipantZ = CoordinatesIntegrated.loc[:,'Agent-Participant_z'].values
AgentParticipantX= CoordinatesIntegrated.loc[:,'Agent-Participant_x'].values

In [17]:
CoordinatesIntegrated['atanPointer'] = [math.atan2(z, x) for z, x in zip(PointerZ , PointerX)]
CoordinatesIntegrated['atanAgent-Part'] = [math.atan2(z, x) for z, x in zip(AgentParticipantZ , AgentParticipantX)]
CoordinatesIntegrated['atanSubs'] = CoordinatesIntegrated['atanPointer'] - CoordinatesIntegrated['atanAgent-Part']
CoordinatesIntegrated['rad2degree']= [np.rad2deg(value) for value in CoordinatesIntegrated['atanSubs']]
signedAngles = CoordinatesIntegrated['rad2degree'].values

In [18]:
for count, angle in enumerate(signedAngles):
    if angle > 180:
        signedAngles[count] = angle - 360
    elif angle < -180:
        signedAngles[count] = angle + 360
CoordinatesIntegrated.insert(11, "SignedAngle-+1802", signedAngles, True)

In [19]:
CoordinatesIntegrated['AbsolutError'] = abs(CoordinatesIntegrated['rad2degree'])
CoordinatesIntegrated[['AbsolutError']].describe()

,AbsolutError
count,2520.000000
mean,56.227750
std,46.741833
min,0.003194
25%,17.376846
50%,43.101340
75%,84.766196
max,179.910652


## Identifying extreme values and flagging first trials

# Flagging the absolute values that transcend 1.5 times and 3 times the inter-quartile range

In [20]:
def find_iqr(x):
    """"
    Returns inter-quartile range of a numeric (i.e, integer or float)
Parameters
-------------
    x: Numeric (i.e, integer or float) variable one wants to calculate interquartile range from
returns
----------
    float that contains interquartile range
    """
    return np.subtract(*np.percentile(x, [75, 25]))


In [21]:
# Create and empty dataframe that will countain interquartile range
inter_quartile_range = pd.DataFrame(columns=['SubjectID', 'IQR'])

#Loop through all subjects and calculate personal IQR
for i, subject in enumerate(CoordinatesIntegrated.SubjectID.unique()):
    inter_quartile_range.at[i, 'SubjectID'] = subject
    inter_quartile_range.at[i, 'IQR'] = find_iqr(CoordinatesIntegrated[CoordinatesIntegrated.SubjectID == subject]['AbsolutError'])

inter_quartile_range['IQRlimit'] = inter_quartile_range['IQR'] * 1.5
inter_quartile_range['IQRlimit2'] = inter_quartile_range['IQR'] * 3

inter_quartile_range.sort_values('IQR')

,SubjectID,IQR,IQRlimit,IQRlimit2
6,5743,42.764168,64.146252,128.292505
0,1031,44.370344,66.555516,133.111031
3,4598,55.165528,82.748292,165.496584
5,5189,56.165709,84.248563,168.497126
12,8629,59.905337,89.858006,179.716012
7,5766,62.246054,93.369082,186.738163
4,4847,63.192045,94.788068,189.576136
8,5972,67.188503,100.782755,201.56551
2,4580,69.568971,104.353457,208.706914
14,9627,72.3025,108.45375,216.9075


In [29]:
# Merged finished data base with inter-quartile range base
interQ = pd.merge(inter_quartile_range,CoordinatesIntegrated, on='SubjectID')
interQ.head(50)

,SubjectID,IQR,IQRlimit,IQRlimit2,Unnamed: 0,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,...,Magnitud_Participant-Pointer,Magnitud_Participant-Building,Magnitud_Participant-Agent,radians,degrees,atanPointer,atanAgent-Part,atanSubs,rad2degree,AbsolutError
0,1031,44.370344,66.555516,133.111031,0,-61.486801,27.881790,-61.211075,27.538521,0.967161,...,68.498927,178.300286,178.300286,1.111540,63.686561,-0.256532,-1.534810,1.278277,73.239902,73.239902
1,1031,44.370344,66.555516,133.111031,1,-228.978378,189.947388,-228.935959,189.613922,0.148005,...,298.254115,382.117633,382.117633,0.403566,23.122601,-1.422214,-1.098297,-0.323917,-18.559074,18.559074
2,1031,44.370344,66.555516,133.111031,2,-114.120316,-139.545837,-114.626694,-140.010651,0.061234,...,179.535248,60.020438,60.020438,1.061713,60.831668,-1.509521,-0.180200,-1.329322,-76.164533,76.164533
3,1031,44.370344,66.555516,133.111031,3,-185.544052,-159.559296,-185.290756,-160.040253,0.876841,...,245.068877,130.800246,130.800246,0.635599,36.417135,-0.501459,0.070825,-0.572284,-32.789460,32.789460
4,1031,44.370344,66.555516,133.111031,41,-61.538162,27.714478,-61.177170,27.703957,0.974709,...,68.291118,320.074036,320.074036,0.428392,24.545025,0.224271,0.014066,0.210205,12.043860,12.043860
5,1031,44.370344,66.555516,133.111031,42,-65.472488,-159.481537,-65.773331,-159.187134,-0.677791,...,172.821362,376.442591,376.442591,0.652186,37.367516,2.316715,0.534296,1.782418,102.125055,102.125055
6,1031,44.370344,66.555516,133.111031,43,-321.010010,77.384674,-320.718018,77.308701,0.982191,...,331.204522,581.271795,581.271795,0.158624,9.088505,-0.188734,-0.077784,-0.110949,-6.356925,6.356925
7,1031,44.370344,66.555516,133.111031,44,-321.092590,77.371643,-320.783142,77.289078,0.961446,...,331.282030,581.353114,581.353114,0.158832,9.100394,-0.278462,-0.077751,-0.200712,-11.499925,11.499925
8,1031,44.370344,66.555516,133.111031,83,-61.534622,27.724136,-61.363297,27.997902,0.123959,...,67.204331,325.786500,325.786500,1.037652,59.453103,1.446402,0.628553,0.817850,46.859332,46.859332
9,1031,44.370344,66.555516,133.111031,120,-61.541702,27.716370,-61.329300,27.528870,0.998588,...,68.428269,97.878324,97.878324,0.597852,34.254371,-0.053065,-1.015725,0.962660,55.156371,55.156371


In [30]:
## Add flag that's true if the puntuaction should stay false if it should go
interQ['Flag'] =  [limit > absolut  for absolut,limit in zip(interQ['AbsolutError'],interQ['IQRlimit'])]
interQ['Flag2'] =  [limit > absolut  for absolut,limit in zip(interQ['AbsolutError'],interQ['IQRlimit2'])]
interQ.head(50)

,SubjectID,IQR,IQRlimit,IQRlimit2,Unnamed: 0,ParticipantPosition_x,ParticipantPosition_z,PointerPosition_x,PointerPosition_z,PointerDirection_x,...,Magnitud_Participant-Agent,radians,degrees,atanPointer,atanAgent-Part,atanSubs,rad2degree,AbsolutError,Flag,Flag2
0,1031,44.370344,66.555516,133.111031,0,-61.486801,27.881790,-61.211075,27.538521,0.967161,...,178.300286,1.111540,63.686561,-0.256532,-1.534810,1.278277,73.239902,73.239902,False,True
1,1031,44.370344,66.555516,133.111031,1,-228.978378,189.947388,-228.935959,189.613922,0.148005,...,382.117633,0.403566,23.122601,-1.422214,-1.098297,-0.323917,-18.559074,18.559074,True,True
2,1031,44.370344,66.555516,133.111031,2,-114.120316,-139.545837,-114.626694,-140.010651,0.061234,...,60.020438,1.061713,60.831668,-1.509521,-0.180200,-1.329322,-76.164533,76.164533,False,True
3,1031,44.370344,66.555516,133.111031,3,-185.544052,-159.559296,-185.290756,-160.040253,0.876841,...,130.800246,0.635599,36.417135,-0.501459,0.070825,-0.572284,-32.789460,32.789460,True,True
4,1031,44.370344,66.555516,133.111031,41,-61.538162,27.714478,-61.177170,27.703957,0.974709,...,320.074036,0.428392,24.545025,0.224271,0.014066,0.210205,12.043860,12.043860,True,True
5,1031,44.370344,66.555516,133.111031,42,-65.472488,-159.481537,-65.773331,-159.187134,-0.677791,...,376.442591,0.652186,37.367516,2.316715,0.534296,1.782418,102.125055,102.125055,False,True
6,1031,44.370344,66.555516,133.111031,43,-321.010010,77.384674,-320.718018,77.308701,0.982191,...,581.271795,0.158624,9.088505,-0.188734,-0.077784,-0.110949,-6.356925,6.356925,True,True
7,1031,44.370344,66.555516,133.111031,44,-321.092590,77.371643,-320.783142,77.289078,0.961446,...,581.353114,0.158832,9.100394,-0.278462,-0.077751,-0.200712,-11.499925,11.499925,True,True
8,1031,44.370344,66.555516,133.111031,83,-61.534622,27.724136,-61.363297,27.997902,0.123959,...,325.786500,1.037652,59.453103,1.446402,0.628553,0.817850,46.859332,46.859332,True,True
9,1031,44.370344,66.555516,133.111031,120,-61.541702,27.716370,-61.329300,27.528870,0.998588,...,97.878324,0.597852,34.254371,-0.053065,-1.015725,0.962660,55.156371,55.156371,True,True


In [31]:
interQ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520 entries, 0 to 2519
Data columns (total 61 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SubjectID                      2520 non-null   object 
 1   IQR                            2520 non-null   object 
 2   IQRlimit                       2520 non-null   object 
 3   IQRlimit2                      2520 non-null   object 
 4   Unnamed: 0                     2520 non-null   int64  
 5   ParticipantPosition_x          2520 non-null   float64
 6   ParticipantPosition_z          2520 non-null   float64
 7   PointerPosition_x              2520 non-null   float64
 8   PointerPosition_z              2520 non-null   float64
 9   PointerDirection_x             2520 non-null   float64
 10  PointerDirection_z             2520 non-null   float64
 11  AgentCenterWorld_z             2520 non-null   float64
 12  AgentCenterWorld_x             2520 non-null   f

In [32]:
interQ.Flag.value_counts()

True     2053
False     467
Name: Flag, dtype: int64

In [33]:
interQ.Flag2.value_counts()

True     2498
False      22
Name: Flag2, dtype: int64

In [34]:
CoordinatesIntegrated.columns

Index(['Unnamed: 0', 'SubjectID', 'ParticipantPosition_x',
       'ParticipantPosition_z', 'PointerPosition_x', 'PointerPosition_z',
       'PointerDirection_x', 'PointerDirection_z', 'AgentCenterWorld_z',
       'AgentCenterWorld_x', 'DistanceToParticipant', 'SignedAngle-+1802',
       'SignedAngle-+180', 'SignedAngle360', 'PointingTaskStartingLocations',
       'AgentID', 'Name', 'StartPointID', 'PointingTargetCondition', 'Context',
       'Agent', 'avatarID', 'AvatarPresence', 'AvatarPresenceCategory',
       'public', 'BuildingCategory', 'activeAgent', 'AgentCategory', 'Trials',
       'TrialNumber', 'StartPointID.1', 'ID_for_StartingPosition', 'ImageName',
       'TimeDelta', 'RT', 'TimeOut', 'TimeStampBegin', 'TimeStampEnd',
       'vector_PartPos', 'vector_AgentPos', 'vector_PointerPos',
       'Vector_Participant-Agent', 'Vector_Participant-Pointer',
       'Agent-Participant_x', 'Agent-Participant_z',
       'Dot_Pointer-Agent-Participant', 'Magnitud_Participant-Pointer',
    

## First pointing trial

In our design the participants have to point to targets from 28 starting locations, they switch positions every 12 trials. It would be sensible to assume that the first trial in each location can count more variance of error since they are starting to familiarize themselves with the environment.

In [35]:
# Flag the first trial of every starting position.
# This has the limitation that it wrongly flags for the last trial of first pointing position
interQ['Removefirst'] = [not (value % 6 == 0) for value in interQ['TrialNumber']]
# The not operator it's used in this case so that all trials that stay are true and the ones to be eliminated are false
interQ[['TrialNumber', 'Removefirst']].head(13)

,TrialNumber,Removefirst
0,1,True
1,67,True
2,110,True
3,127,True
4,2,True
5,54,False
6,166,True
7,168,False
8,3,True
9,4,True


In [36]:
# Here we correct for the mistake the previous method created (i.e., deflag the 12th trial)
interQ.loc[interQ['TrialNumber'] == 6, 'Removefirst'] = True
# And we flag all first trials
interQ.loc[interQ['TrialNumber'] == 1, 'Removefirst'] = False

In [37]:
# Proof of concept: Here we retain only the flagged trials and check if the correct Trial Ids have been identified
TeoryFirst = interQ[interQ['Removefirst'] == False].copy()
check = pd.DataFrame(TeoryFirst['TrialNumber'].unique())
check.describe()

,0
count,28.000000
mean,86.821429
std,49.667825
min,1.000000
25%,46.500000
50%,87.000000
75%,127.500000
max,168.000000


In [38]:
# Added cell to set Working Directory to your location
os.chdir('E:/HumanA/Data/Data_Tracy')
#os.chdir('/Users/tracysanchezpacheco/Documents/Resources')
os.getcwd()
interQ.to_csv('AgentPointingTask_IQR_Final.csv')

In [39]:
interQ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2520 entries, 0 to 2519
Data columns (total 62 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   SubjectID                      2520 non-null   object 
 1   IQR                            2520 non-null   object 
 2   IQRlimit                       2520 non-null   object 
 3   IQRlimit2                      2520 non-null   object 
 4   Unnamed: 0                     2520 non-null   int64  
 5   ParticipantPosition_x          2520 non-null   float64
 6   ParticipantPosition_z          2520 non-null   float64
 7   PointerPosition_x              2520 non-null   float64
 8   PointerPosition_z              2520 non-null   float64
 9   PointerDirection_x             2520 non-null   float64
 10  PointerDirection_z             2520 non-null   float64
 11  AgentCenterWorld_z             2520 non-null   float64
 12  AgentCenterWorld_x             2520 non-null   f

In [40]:
#Elimininating trails were the participant ran out of time
Clean = interQ[interQ['TimeOut']== False]

In [41]:
PointingSum = pd.DataFrame(Clean[['SubjectID', 'AbsolutError', 'SignedAngle-+180', 'IQR', 'RT',  'DistanceToParticipant', 'PointingTaskStartingLocations',  
                                  'TrialNumber', 'StartPointID', 'ID_for_StartingPosition','avatarID', 'AvatarPresenceCategory', 'Context', 'BuildingCategory',  
                                  'AgentCategory','ImageName']].values)

In [42]:
PointingSum.columns = ['ID', 'AbsolutError', 'SignedAngle-+180', 'IQR', 'RT',  'DistanceToParticipant', 'PointingTaskStartingLocations',  'TrialNumber', 
                       'StartPointID', 'ID_for_StartingPosition','avatarID', 'AvatarPresenceCategory', 'Context', 'BuildingCategory','AgentCategory',  'ImageName']
PointingSum.head()

,ID,AbsolutError,SignedAngle-+180,IQR,RT,DistanceToParticipant,PointingTaskStartingLocations,TrialNumber,StartPointID,ID_for_StartingPosition,avatarID,AvatarPresenceCategory,Context,BuildingCategory,AgentCategory,ImageName
0,1031,73.239902,73.239902,44.370344,9.057461,172.425797,15,1,1,1,16,Present,False,Residential,Active,16_SaAct_92_36
1,1031,18.559074,-18.559074,44.370344,28.543825,378.171112,23,67,12,1,16,Present,False,Residential,Active,16_SaAct_92_36
2,1031,76.164533,-76.164533,44.370344,5.412356,61.858631,21,110,19,2,16,Present,False,Residential,Active,16_SaAct_92_36
3,1031,32.78946,-32.78946,44.370344,8.281719,133.832611,4,127,22,1,16,Present,False,Residential,Active,16_SaAct_92_36
4,1031,12.04386,12.04386,44.370344,3.833091,321.284546,15,2,1,2,50,Present,False,Residential,Passive,50_Sa_66_15


In [43]:
os.chdir('E:/HumanA/Data/Data_Tracy')
#os.chdir('/Users/tracysanchezpacheco/Documents/Resources')
PointingSum.to_csv('AgentPointingSummary.csv')
Clean.to_csv('FULL_AgentPointing.csv')

In [44]:
PointingSum[['ImageName', 'avatarID']]

,ImageName,avatarID
0,16_SaAct_92_36,16
1,16_SaAct_92_36,16
2,16_SaAct_92_36,16
3,16_SaAct_92_36,16
4,50_Sa_66_15,50
...,...,...
2515,54_Sa_80_26,54
2516,54_Sa_80_26,54
2517,54_Sa_80_26,54
2518,47_Sa_74_22,47
